In [6]:
import pandas as pd
from scipy.stats import zscore
import seaborn as sns
from scipy.cluster.hierarchy import fcluster
import numpy as np

In [7]:
header = ['Naive 1', 'Naive 2', 'Naive 3', 'Naive 4', 'Early Pre-TFH 1', 'Early Pre-TFH 2', 'Early Pre-TFH 3', 'Early Pre-TFH 4', 'Late Pre-TFH 1', 'Late Pre-TFH 2', 'Late Pre-TFH 3', 'Late Pre-TFH 4', 'GC 1', 'GC 2', 'GC 3', 'GC 4']

rna_seq = pd.read_csv('/ix/djishnu/Alisa/Tfh/Run_313.atac-seq-correl.rsem.pe.hg38.ensembl.counts.matrix.results.normalized_data_matrix', skiprows = [0], names=header, sep ='\t')
rna_seq = rna_seq.reset_index()

rna_seq = rna_seq.rename(columns={'index': "Gene"})
rna_seq[['Gene_ID', 'Genes']] = rna_seq.Gene.str.split("_",n=1, expand=True)

In [8]:
rna_seq['ATAC_Naive'] = rna_seq[['Naive 1', 'Naive 2', 'Naive 3', 'Naive 4']].mean(axis=1) + 0.000001
rna_seq['ATAC_Early_Pre-TFH'] = rna_seq[['Early Pre-TFH 1', 'Early Pre-TFH 2', 'Early Pre-TFH 3', 'Early Pre-TFH 4']].mean(axis=1) + 0.000001
rna_seq['ATAC_Late_Pre-TFH'] = rna_seq[['Late Pre-TFH 1', 'Late Pre-TFH 2', 'Late Pre-TFH 3', 'Late Pre-TFH 4']].mean(axis=1) + 0.000001
rna_seq['ATAC_GC'] = rna_seq[['GC 1', 'GC 2', 'GC 3', 'GC 4']].mean(axis=1) + 0.000001
rna_seq_mean = rna_seq[['Genes', 'ATAC_Naive','ATAC_Early_Pre-TFH', 'ATAC_Late_Pre-TFH', 'ATAC_GC']]

In [9]:
rna_seq_mean = rna_seq_mean.set_index('Genes')

In [24]:
rna_seq_mean['LogChange'] = np.log2(np.divide(rna_seq_mean["ATAC_GC"],rna_seq_mean["ATAC_Early_Pre-TFH"]))
rna_seq_mean = rna_seq_mean.reset_index()

#rna_seq_mean_filtered = rna_seq_mean[rna_seq_mean['total'] > threshold]
#print(rna_seq_mean_filtered)
rna_seq_sign_change = rna_seq_mean[['Genes', 'LogChange']]
print(rna_seq_sign_change)

               Genes  LogChange
0             TSPAN6 -17.554026
1               TNMD   0.000000
2               DPM1  -0.131166
3              SCYL3   2.485725
4           C1orf112  -2.487404
...              ...        ...
60670  CTD-2060L22.1 -16.784421
60671   RP11-107E5.4   0.000000
60672          HYMAI  20.806715
60673     RARRES2P11   0.000000
60674   RP11-299P2.2   0.000000

[60675 rows x 2 columns]


In [25]:
#Check direction:
print(rna_seq_mean[rna_seq_mean['Genes']=='BCL6'])

      index Genes  ATAC_Naive  ATAC_Early_Pre-TFH  ATAC_Late_Pre-TFH  \
4239   4239  BCL6   66.066535           91.347667         358.027455   

         ATAC_GC  LogChange  
4239  403.335255    2.14254  


In [26]:
rna_seq_sign_change.to_csv('../../Sample_outputs/correct_direction_sign_change_rna_earlyvsGC_log2FC.csv')

In [27]:
#Cap the LogFC values at abs(2):
rna_seq_sign_change = rna_seq_sign_change.copy()
rna_seq_sign_change['LogChange_Thresh'] = rna_seq_sign_change['LogChange'].clip(lower=-2, upper=2)



In [28]:
rna_seq_sign_change.to_csv('../../Sample_outputs/correct_direction_sign_change_rna_earlyvsGC_Thresh_log2FC.csv')

In [30]:
# Make a safe copy to avoid modifying the original DataFrame
rna_seq_filtered = rna_seq_mean.copy()

# Compute row means only on numeric columns
rna_seq_filtered["total"] = rna_seq_filtered.mean(axis=1, numeric_only=True)

# Calculate the 10% threshold
threshold = rna_seq_filtered["total"].quantile(0.10)

# Filter out the lowest 10%
filtered_df = rna_seq_filtered[rna_seq_filtered["total"] > threshold]


In [32]:
# Function to map column names to scores
def map_to_scores(column_name):
    column_mapping = {
        'ATAC_Naive': 0,
        'ATAC_Early_Pre-TFH': 1,
        'ATAC_Late_Pre-TFH': 2,
        'ATAC_GC': 3
    }
    return column_mapping[column_name]

# Select only the numeric expression columns for idxmax
expression_cols = ['ATAC_Naive', 'ATAC_Early_Pre-TFH', 'ATAC_Late_Pre-TFH', 'ATAC_GC']
max_columns = filtered_df[expression_cols].idxmax(axis=1)

# Map the column names to scores and create the State column
filtered_df = filtered_df.copy()
filtered_df['State'] = max_columns.apply(map_to_scores)

# Save to CSV
filtered_df.to_csv('../../Sample_outputs/rna_state_specific.csv')

print(filtered_df)


       index          Genes   ATAC_Naive  ATAC_Early_Pre-TFH  \
1316    1316           ACTB  8422.419944         9197.528240   
1822    1822          RPLP0  1695.126741         1170.278780   
2261    2261           RPL3  4648.864158         3161.038351   
3149    3149         RPL18A  1720.034289         1080.773323   
3506    3506          RPL19  2277.307876         1653.059116   
...      ...            ...          ...                 ...   
60670  60670  CTD-2060L22.1     0.000001            0.112879   
60671  60671   RP11-107E5.4     0.000001            0.000001   
60672  60672          HYMAI     0.000001            0.000001   
60673  60673     RARRES2P11     0.000001            0.000001   
60674  60674   RP11-299P2.2     0.000001            0.000001   

       ATAC_Late_Pre-TFH       ATAC_GC  LogChange         total  State  
1316         8743.239117  12151.277559   0.401790   6638.477775      3  
1822         1296.100210    732.030630  -0.676876   1119.143247      0  
2261        